# Limpieza archivo CSV (Fuente: Tripadvisor):

Una vez tenemos el archivos csv,procedemos a limpiarlo.

In [1]:
import pandas as pd

import sys
sys.path.append("..")
from src.funciones import * 

In [2]:
ruta_al_csv = '../data/Tripadvisor.csv'
df = pd.read_csv(ruta_al_csv)
data=df.copy()

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95 entries, 0 to 94
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       95 non-null     object
 1   1       95 non-null     object
 2   2       95 non-null     object
 3   3       12 non-null     object
 4   4       2 non-null      object
dtypes: object(5)
memory usage: 3.8+ KB


In [4]:
data.describe().T

,count,unique,top,freq
0,95,59,MICHELIN,7
1,95,61,Viajes,5
2,95,28,"Italiana, Mediterránea€€ - €€€Carta",18
3,12,5,"Japonesa, Sushi€€€€Carta",4
4,2,1,Iniciar sesión,2


In [5]:
data.head()

,0,1,2,3,4
0,Descubrir,Viajes,Comunidad,Iniciar sesión,NaN
1,1. Pizzart Fuencarral,897 opinionesAbierto ahora,"Italiana, Pizza€€ - €€€Carta",NaN,NaN
2,2. Maximo's,267 opinionesAbierto ahora,"Japonesa, Sushi€€ - €€€Carta",NaN,NaN
3,3. Ornella Velázquez,2282 opinionesAbierto ahora,"Italiana, Mediterránea€€ - €€€Carta",NaN,NaN
4,4. Smok Mok,103 opinionesAbierto ahora,"Japonesa, Asiática€",NaN,NaN


Tras ver el dataframe procedemos con la siguiente limpieza:

* Ponemos los títulos bien
* Quitamos las filas que han salido mal durante la extracción de datos
* Quitamos los números que preceden a los nomber
* Reseteamos los índices
* Quitamos de la columna opiniones Abierto/Cerrado ahora
* Quitamos de Tipo '€' y los añadimos a precio la equivalencia
* Quitamos Carta de Tipo

In [6]:
data.columns=['Nombre','Opiniones','Tipo','Precio','Cosas']

eliminar=['Patrocinado', 'Descubrir','MICHELIN','Opinión']
data = data[~data['Nombre'].isin(eliminar)]

data['Nombre'] = data['Nombre'].str.replace(r'\d+\.\s', '', regex=True) #eliminamos numeros de nombre
data = data.reset_index(drop=True)

data['Opiniones'] = data['Opiniones'].str.replace('Abierto ahora', '')
data['Opiniones'] = data['Opiniones'].str.replace('Cerrado ahora', '')

data['Tipo'] = data['Tipo'].str.replace(r'Carta$', '', regex=True)
data['Precio'] = data['Tipo'].str.extract(r'(€+)')

In [7]:
def precio(tipo):
    if '€€€€' in tipo:
        return 'Alto'
    elif '€€ - €€€' in tipo:
        return 'Medio'
    elif '€' in tipo:
        return 'Bajo'
    else:
        return ''

In [8]:
data['Precio'] = data['Tipo'].apply(precio)
data['Tipo'] = data['Tipo'].str.replace('€', '', regex=False)
data['Tipo'] = data['Tipo'].str.replace('-', '', regex=False)

In [9]:
data.head()

,Nombre,Opiniones,Tipo,Precio,Cosas
0,Pizzart Fuencarral,897 opiniones,"Italiana, Pizza",Medio,NaN
1,Maximo's,267 opiniones,"Japonesa, Sushi",Medio,NaN
2,Ornella Velázquez,2282 opiniones,"Italiana, Mediterránea",Medio,NaN
3,Smok Mok,103 opiniones,"Japonesa, Asiática",Bajo,NaN
4,Maruzzella,960 opiniones,"Italiana, Napolitana",Medio,NaN


Ponemos valores únicos en Tipo:

In [10]:
limpieza_tipo(data)

In [11]:
data.Tipo.value_counts()

Tipo
Italiana      51
Japonesa      12
India          6
China          4
Japonesa       3
China          3
Mexicana       2
Sushi          2
Name: count, dtype: int64

In [12]:
data.tail()

,Nombre,Opiniones,Tipo,Precio,Cosas
78,Art & Sushi,288 opiniones,Sushi,Medio,NaN
79,That's Amore,55 opiniones,Italiana,Medio,NaN
80,Bowlers Kitchen,49 opiniones,India,Medio,NaN
81,Lamonarracha-Chamberí,91 opiniones,Japonesa,Medio,NaN
82,Vecchia Milano,133 opiniones,Italiana,Medio,NaN
